# Anomaly Detection Experiment Template

**Purpose**: Comprehensive template for anomaly detection experiments using Pynomaly

**Version**: 1.0.0

**Created**: 2025-06-24

**Author**: [Your Name]

---

## 📋 Experiment Overview

This notebook provides a standardized framework for conducting anomaly detection experiments. It includes:

- 📊 **Data exploration and quality assessment**
- 🔧 **Data preprocessing and feature engineering**
- 🤖 **Multiple algorithm comparison**
- 📈 **Performance evaluation and statistical testing**
- 📊 **Comprehensive visualization and reporting**
- 🔄 **Reproducible experiment workflow**

### 🎯 Experiment Objectives

1. [Define your primary objective here]
2. [Define your secondary objective here]
3. [Define success criteria here]

### 📝 Experiment Metadata

- **Dataset**: [Dataset name and description]
- **Target**: [What you're trying to detect]
- **Algorithms**: [List of algorithms to compare]
- **Evaluation**: [Primary evaluation metric]
- **Timeline**: [Expected duration]

## 🛠️ Setup and Configuration

In [ ]:
# Standard imports
import warnings
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

# Scientific computing
import os

# Pynomaly imports
import sys

from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "src"))

from pynomaly.domain.entities.dataset import Dataset
from pynomaly.domain.value_objects.contamination_rate import ContaminationRate
from pynomaly.infrastructure.adapters.sklearn_adapter import SklearnAdapter

# Visualization setup
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12

# Random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Setup complete")
print(f"📅 Experiment started: {datetime.now()}")

## 📂 Data Loading and Initial Exploration

In [ ]:
# Configuration
DATA_PATH = "data/your_dataset.csv"  # Update this path
TARGET_COLUMN = "is_anomaly"  # Update if different
ID_COLUMNS = ["id"]  # Columns to exclude from features

# Load data
try:
    df = pd.read_csv(DATA_PATH)
    print(f"✅ Data loaded successfully: {df.shape}")
except FileNotFoundError:
    print(f"❌ File not found: {DATA_PATH}")
    print("Please update DATA_PATH with the correct file path")
    # Create sample data for demonstration
    from sklearn.datasets import make_classification

    X, y = make_classification(
        n_samples=1000,
        n_features=10,
        n_clusters_per_class=1,
        n_redundant=0,
        random_state=RANDOM_STATE,
    )
    df = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(10)])
    df[TARGET_COLUMN] = y
    print(f"📊 Using sample data: {df.shape}")

# Display basic information
print(f"\n📊 Dataset shape: {df.shape}")
print(f"📋 Columns: {list(df.columns)}")
print(f"🎯 Target column: {TARGET_COLUMN}")

# Check if target exists
if TARGET_COLUMN in df.columns:
    print("\n🎯 Target distribution:")
    print(df[TARGET_COLUMN].value_counts())
    print(f"📊 Anomaly rate: {df[TARGET_COLUMN].mean():.1%}")
else:
    print(
        f"⚠️ Target column '{TARGET_COLUMN}' not found - assuming unsupervised learning"
    )
    TARGET_COLUMN = None

# Display first few rows
display(df.head())

## 🔍 Data Quality Assessment

In [ ]:
def assess_data_quality(df):
    """Comprehensive data quality assessment."""
    print("=== DATA QUALITY ASSESSMENT ===")

    # Basic statistics
    print("\n📊 Basic Statistics:")
    print(f"  • Total rows: {len(df):,}")
    print(f"  • Total columns: {len(df.columns)}")
    print(f"  • Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    # Missing values
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_summary = pd.DataFrame(
        {"Missing Count": missing, "Missing %": missing_pct}
    ).sort_values("Missing %", ascending=False)

    print("\n❓ Missing Values:")
    if missing.sum() > 0:
        print(missing_summary[missing_summary["Missing Count"] > 0])
    else:
        print("  ✅ No missing values found")

    # Duplicates
    duplicates = df.duplicated().sum()
    print(f"\n🔄 Duplicate rows: {duplicates} ({duplicates / len(df) * 100:.1f}%)")

    # Data types
    print("\n📋 Data Types:")
    dtype_summary = df.dtypes.value_counts()
    for dtype, count in dtype_summary.items():
        print(f"  • {dtype}: {count} columns")

    # Numeric columns analysis
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print("\n📈 Numeric Columns Summary:")
        display(df[numeric_cols].describe().round(3))

    # Categorical columns analysis
    categorical_cols = df.select_dtypes(include=["object"]).columns
    if len(categorical_cols) > 0:
        print("\n📝 Categorical Columns:")
        for col in categorical_cols:
            unique_count = df[col].nunique()
            print(f"  • {col}: {unique_count} unique values")
            if unique_count <= 10:
                print(f"    Values: {list(df[col].unique())}")

    return missing_summary, duplicates


# Perform assessment
missing_summary, duplicates = assess_data_quality(df)

## 📊 Exploratory Data Analysis (EDA)

In [ ]:
# Correlation analysis
numeric_cols = df.select_dtypes(include=[np.number]).columns
if TARGET_COLUMN:
    numeric_cols = [col for col in numeric_cols if col != TARGET_COLUMN]

if len(numeric_cols) > 1:
    # Correlation matrix
    plt.figure(figsize=(12, 10))
    correlation_matrix = df[numeric_cols].corr()

    # Create heatmap
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(
        correlation_matrix,
        mask=mask,
        annot=True,
        cmap="coolwarm",
        center=0,
        square=True,
        linewidths=0.5,
        cbar_kws={"shrink": 0.5},
    )
    plt.title("Feature Correlation Matrix", fontsize=16, fontweight="bold")
    plt.tight_layout()
    plt.show()

    # High correlation pairs
    high_corr_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i + 1, len(correlation_matrix.columns)):
            corr_val = correlation_matrix.iloc[i, j]
            if abs(corr_val) > 0.8:
                high_corr_pairs.append(
                    (
                        correlation_matrix.columns[i],
                        correlation_matrix.columns[j],
                        corr_val,
                    )
                )

    if high_corr_pairs:
        print("🔗 High correlation pairs (|r| > 0.8):")
        for col1, col2, corr in high_corr_pairs:
            print(f"  • {col1} ↔ {col2}: {corr:.3f}")
    else:
        print("✅ No highly correlated feature pairs found")

In [ ]:
# Distribution analysis
if len(numeric_cols) > 0:
    # Select up to 6 most important features for visualization
    viz_cols = numeric_cols[:6]

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle("Feature Distributions", fontsize=16, fontweight="bold")

    axes = axes.flatten()

    for i, col in enumerate(viz_cols):
        if i < len(axes):
            ax = axes[i]

            # Plot distribution
            if TARGET_COLUMN and TARGET_COLUMN in df.columns:
                # Separate by target class
                normal_data = df[df[TARGET_COLUMN] == 0][col].dropna()
                anomaly_data = df[df[TARGET_COLUMN] == 1][col].dropna()

                ax.hist(normal_data, bins=30, alpha=0.7, label="Normal", density=True)
                ax.hist(anomaly_data, bins=30, alpha=0.7, label="Anomaly", density=True)
                ax.legend()
            else:
                # Single distribution
                ax.hist(df[col].dropna(), bins=30, alpha=0.7, density=True)

            ax.set_title(f"{col}", fontweight="bold")
            ax.set_ylabel("Density")
            ax.grid(True, alpha=0.3)

    # Hide unused subplots
    for i in range(len(viz_cols), len(axes)):
        axes[i].set_visible(False)

    plt.tight_layout()
    plt.show()

## 🔧 Data Preprocessing

In [ ]:
def preprocess_data(df, target_column=None, id_columns=None):
    """Comprehensive data preprocessing pipeline."""
    print("=== DATA PREPROCESSING ===")

    # Make a copy
    df_processed = df.copy()

    # Remove ID columns
    if id_columns:
        id_cols_present = [col for col in id_columns if col in df_processed.columns]
        if id_cols_present:
            df_processed = df_processed.drop(columns=id_cols_present)
            print(f"🗑️ Removed ID columns: {id_cols_present}")

    # Separate features and target
    if target_column and target_column in df_processed.columns:
        y = df_processed[target_column]
        X = df_processed.drop(columns=[target_column])
        print(f"🎯 Target separated: {target_column}")
    else:
        y = None
        X = df_processed
        print("📊 Unsupervised mode: no target column")

    # Handle missing values
    missing_before = X.isnull().sum().sum()
    if missing_before > 0:
        print(f"❓ Handling {missing_before} missing values...")

        # Numeric columns: fill with median
        numeric_cols = X.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            if X[col].isnull().any():
                median_val = X[col].median()
                X[col].fillna(median_val, inplace=True)
                print(f"  • {col}: filled with median ({median_val:.3f})")

        # Categorical columns: fill with mode
        categorical_cols = X.select_dtypes(include=["object"]).columns
        for col in categorical_cols:
            if X[col].isnull().any():
                mode_val = (
                    X[col].mode().iloc[0] if not X[col].mode().empty else "Unknown"
                )
                X[col].fillna(mode_val, inplace=True)
                print(f"  • {col}: filled with mode ({mode_val})")

    # Encode categorical variables
    categorical_cols = X.select_dtypes(include=["object"]).columns
    if len(categorical_cols) > 0:
        print(f"🏷️ Encoding {len(categorical_cols)} categorical columns...")

        for col in categorical_cols:
            unique_count = X[col].nunique()
            if unique_count <= 10:  # One-hot encode low cardinality
                dummies = pd.get_dummies(X[col], prefix=col, drop_first=True)
                X = pd.concat([X.drop(columns=[col]), dummies], axis=1)
                print(f"  • {col}: one-hot encoded ({unique_count} categories)")
            else:  # Label encode high cardinality
                le = LabelEncoder()
                X[col] = le.fit_transform(X[col])
                print(f"  • {col}: label encoded ({unique_count} categories)")

    # Scale numeric features
    numeric_cols = X.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(f"📏 Scaling {len(numeric_cols)} numeric features...")
        scaler = StandardScaler()
        X[numeric_cols] = scaler.fit_transform(X[numeric_cols])
        print("  ✅ StandardScaler applied")

    print("\n✅ Preprocessing complete")
    print(f"  • Final shape: {X.shape}")
    print(f"  • Feature columns: {len(X.columns)}")

    return X, y


# Apply preprocessing
X, y = preprocess_data(df, TARGET_COLUMN, ID_COLUMNS)

# Display processed data info
print("\n📊 Processed data summary:")
print(f"  • Features shape: {X.shape}")
if y is not None:
    print(f"  • Target shape: {y.shape}")
    print(f"  • Target distribution: {y.value_counts().to_dict()}")

# Show sample of processed features
display(X.head())

## 🤖 Algorithm Setup and Configuration

In [ ]:
# Algorithm configuration
CONTAMINATION_RATE = 0.1  # Expected proportion of anomalies
ALGORITHMS = {
    "IsolationForest": {
        "adapter": SklearnAdapter,
        "params": {
            "contamination": CONTAMINATION_RATE,
            "n_estimators": 100,
            "random_state": RANDOM_STATE,
        },
    },
    "LocalOutlierFactor": {
        "adapter": SklearnAdapter,
        "params": {
            "contamination": CONTAMINATION_RATE,
            "n_neighbors": 20,
            "novelty": True,
        },
    },
    "OneClassSVM": {
        "adapter": SklearnAdapter,
        "params": {"nu": CONTAMINATION_RATE, "kernel": "rbf", "gamma": "scale"},
    },
}

print(f"🤖 Configured {len(ALGORITHMS)} algorithms:")
for name, config in ALGORITHMS.items():
    print(f"  • {name}: {config['params']}")

# Create Pynomaly dataset
dataset = Dataset(name="experiment_dataset", data=X, feature_names=list(X.columns))

print(f"\n📊 Dataset created: {dataset.summary()}")

## 🔬 Experiment Execution

In [ ]:
# Initialize results storage
results = {}
execution_times = {}

print("=== EXPERIMENT EXECUTION ===")
print(f"🚀 Starting experiment with {len(ALGORITHMS)} algorithms...\n")

for alg_name, config in ALGORITHMS.items():
    print(f"🔄 Running {alg_name}...")

    try:
        # Start timing
        start_time = datetime.now()

        # Create contamination rate object
        contamination_rate = ContaminationRate(CONTAMINATION_RATE)

        # Initialize adapter
        if config["adapter"] == SklearnAdapter:
            adapter = SklearnAdapter(alg_name, contamination_rate=contamination_rate)
        else:
            adapter = config["adapter"](alg_name, contamination_rate=contamination_rate)

        # Fit and detect
        result = adapter.fit_detect(dataset)

        # Record execution time
        end_time = datetime.now()
        execution_time = (end_time - start_time).total_seconds()
        execution_times[alg_name] = execution_time

        # Extract scores and predictions
        scores = np.array([score.value for score in result.scores])
        predictions = (
            scores > np.percentile(scores, (1 - CONTAMINATION_RATE) * 100)
        ).astype(int)

        # Store results
        results[alg_name] = {
            "scores": scores,
            "predictions": predictions,
            "result_object": result,
            "execution_time": execution_time,
        }

        print(f"  ✅ {alg_name} completed in {execution_time:.2f}s")
        print(f"     Detected {predictions.sum()} anomalies ({predictions.mean():.1%})")

    except Exception as e:
        print(f"  ❌ {alg_name} failed: {str(e)}")
        results[alg_name] = {"error": str(e)}

print(
    f"\n🎯 Experiment completed! {len([r for r in results.values() if 'error' not in r])} algorithms successful."
)

## 📊 Performance Evaluation

In [ ]:
# Performance evaluation (if ground truth is available)
if y is not None:
    from sklearn.metrics import (
        average_precision_score,
        f1_score,
        precision_score,
        recall_score,
        roc_auc_score,
    )

    print("=== PERFORMANCE EVALUATION ===")

    # Create performance summary
    performance_df = pd.DataFrame()

    for alg_name, result in results.items():
        if "error" in result:
            continue

        predictions = result["predictions"]
        scores = result["scores"]

        # Calculate metrics
        metrics = {
            "Algorithm": alg_name,
            "Precision": precision_score(y, predictions),
            "Recall": recall_score(y, predictions),
            "F1-Score": f1_score(y, predictions),
            "ROC-AUC": roc_auc_score(y, scores),
            "PR-AUC": average_precision_score(y, scores),
            "Execution Time (s)": result["execution_time"],
        }

        performance_df = pd.concat(
            [performance_df, pd.DataFrame([metrics])], ignore_index=True
        )

    # Display performance table
    if not performance_df.empty:
        print("📊 Performance Summary:")
        display(performance_df.round(4))

        # Identify best algorithms
        best_f1 = performance_df.loc[performance_df["F1-Score"].idxmax(), "Algorithm"]
        best_auc = performance_df.loc[performance_df["ROC-AUC"].idxmax(), "Algorithm"]
        fastest = performance_df.loc[
            performance_df["Execution Time (s)"].idxmin(), "Algorithm"
        ]

        print("\n🏆 Best Performers:")
        print(
            f"  • Best F1-Score: {best_f1} ({performance_df[performance_df['Algorithm'] == best_f1]['F1-Score'].iloc[0]:.4f})"
        )
        print(
            f"  • Best ROC-AUC: {best_auc} ({performance_df[performance_df['Algorithm'] == best_auc]['ROC-AUC'].iloc[0]:.4f})"
        )
        print(
            f"  • Fastest: {fastest} ({performance_df[performance_df['Algorithm'] == fastest]['Execution Time (s)'].iloc[0]:.2f}s)"
        )

else:
    print("⚠️ No ground truth available - skipping supervised evaluation")

    # Unsupervised evaluation metrics
    print("=== UNSUPERVISED EVALUATION ===")

    unsupervised_df = pd.DataFrame()

    for alg_name, result in results.items():
        if "error" in result:
            continue

        scores = result["scores"]
        predictions = result["predictions"]

        metrics = {
            "Algorithm": alg_name,
            "Anomalies Detected": predictions.sum(),
            "Anomaly Rate": predictions.mean(),
            "Score Mean": scores.mean(),
            "Score Std": scores.std(),
            "Execution Time (s)": result["execution_time"],
        }

        unsupervised_df = pd.concat(
            [unsupervised_df, pd.DataFrame([metrics])], ignore_index=True
        )

    if not unsupervised_df.empty:
        print("📊 Unsupervised Metrics Summary:")
        display(unsupervised_df.round(4))

## 📈 Visualization and Analysis

In [ ]:
# ROC Curves (if ground truth available)
if y is not None and len([r for r in results.values() if "error" not in r]) > 0:
    plt.figure(figsize=(12, 8))

    colors = ["blue", "red", "green", "orange", "purple"]

    for i, (alg_name, result) in enumerate(results.items()):
        if "error" in result:
            continue

        scores = result["scores"]

        # Calculate ROC curve
        fpr, tpr, _ = roc_curve(y, scores)
        auc_score = roc_auc_score(y, scores)

        # Plot ROC curve
        plt.plot(
            fpr,
            tpr,
            color=colors[i % len(colors)],
            linewidth=2,
            label=f"{alg_name} (AUC = {auc_score:.3f})",
        )

    # Plot diagonal line
    plt.plot([0, 1], [0, 1], "k--", linewidth=1, alpha=0.8)

    plt.xlabel("False Positive Rate", fontsize=12, fontweight="bold")
    plt.ylabel("True Positive Rate", fontsize=12, fontweight="bold")
    plt.title("ROC Curves Comparison", fontsize=16, fontweight="bold")
    plt.legend(loc="lower right", fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])

    plt.tight_layout()
    plt.show()

In [ ]:
# Score distributions comparison
successful_results = {
    name: result for name, result in results.items() if "error" not in result
}

if len(successful_results) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle("Algorithm Comparison Analysis", fontsize=16, fontweight="bold")

    # 1. Score distributions
    ax1 = axes[0, 0]
    for alg_name, result in successful_results.items():
        scores = result["scores"]
        ax1.hist(scores, bins=30, alpha=0.6, label=alg_name, density=True)
    ax1.set_xlabel("Anomaly Score")
    ax1.set_ylabel("Density")
    ax1.set_title("Score Distributions")
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # 2. Execution time comparison
    ax2 = axes[0, 1]
    alg_names = list(successful_results.keys())
    exec_times = [successful_results[name]["execution_time"] for name in alg_names]
    bars = ax2.bar(alg_names, exec_times, alpha=0.7)
    ax2.set_ylabel("Execution Time (seconds)")
    ax2.set_title("Execution Time Comparison")
    ax2.tick_params(axis="x", rotation=45)

    # Add value labels on bars
    for bar, time_val in zip(bars, exec_times, strict=False):
        height = bar.get_height()
        ax2.text(
            bar.get_x() + bar.get_width() / 2.0,
            height + 0.01,
            f"{time_val:.2f}s",
            ha="center",
            va="bottom",
        )

    # 3. Anomaly detection rates
    ax3 = axes[1, 0]
    detection_rates = [
        successful_results[name]["predictions"].mean() for name in alg_names
    ]
    bars = ax3.bar(alg_names, detection_rates, alpha=0.7, color="coral")
    ax3.set_ylabel("Anomaly Detection Rate")
    ax3.set_title("Detection Rate Comparison")
    ax3.tick_params(axis="x", rotation=45)
    ax3.axhline(
        y=CONTAMINATION_RATE,
        color="red",
        linestyle="--",
        label=f"Expected Rate ({CONTAMINATION_RATE:.1%})",
    )
    ax3.legend()

    # Add value labels
    for bar, rate in zip(bars, detection_rates, strict=False):
        height = bar.get_height()
        ax3.text(
            bar.get_x() + bar.get_width() / 2.0,
            height + 0.001,
            f"{rate:.1%}",
            ha="center",
            va="bottom",
        )

    # 4. Performance radar chart (if ground truth available)
    ax4 = axes[1, 1]
    if y is not None and not performance_df.empty:
        # Create radar chart
        metrics = ["Precision", "Recall", "F1-Score", "ROC-AUC", "PR-AUC"]

        # Select up to 3 algorithms for clarity
        top_algorithms = performance_df.nlargest(3, "F1-Score")["Algorithm"].tolist()

        angles = np.linspace(0, 2 * np.pi, len(metrics), endpoint=False).tolist()
        angles += angles[:1]  # Complete the circle

        for i, alg in enumerate(top_algorithms):
            values = (
                performance_df[performance_df["Algorithm"] == alg][metrics]
                .iloc[0]
                .tolist()
            )
            values += values[:1]  # Complete the circle

            ax4.plot(angles, values, "o-", linewidth=2, label=alg)
            ax4.fill(angles, values, alpha=0.25)

        ax4.set_xticks(angles[:-1])
        ax4.set_xticklabels(metrics)
        ax4.set_ylim(0, 1)
        ax4.set_title("Performance Comparison (Top 3)")
        ax4.legend(loc="upper right", bbox_to_anchor=(1.3, 1.0))
        ax4.grid(True)
    else:
        ax4.text(
            0.5,
            0.5,
            "Performance Radar\n(Ground Truth Required)",
            ha="center",
            va="center",
            transform=ax4.transAxes,
            fontsize=12,
            style="italic",
        )
        ax4.set_title("Performance Radar Chart")

    plt.tight_layout()
    plt.show()

## 📋 Statistical Analysis and Significance Testing

In [ ]:
# Statistical significance testing (if ground truth available)
if y is not None and len(successful_results) > 1:
    print("=== STATISTICAL SIGNIFICANCE TESTING ===")

    # Cross-validation for statistical testing
    from scipy.stats import friedmanchisquare, wilcoxon
    from sklearn.model_selection import cross_val_score

    # Collect cross-validation scores
    cv_scores = {}

    print("🔄 Performing 5-fold cross-validation...")

    for alg_name in successful_results.keys():
        try:
            # Create sklearn-compatible estimator for CV
            if alg_name == "IsolationForest":
                from sklearn.ensemble import IsolationForest

                estimator = IsolationForest(
                    contamination=CONTAMINATION_RATE, random_state=RANDOM_STATE
                )
            elif alg_name == "LocalOutlierFactor":
                from sklearn.neighbors import LocalOutlierFactor

                estimator = LocalOutlierFactor(
                    contamination=CONTAMINATION_RATE, novelty=True
                )
            elif alg_name == "OneClassSVM":
                from sklearn.svm import OneClassSVM

                estimator = OneClassSVM(nu=CONTAMINATION_RATE)
            else:
                continue

            # Perform cross-validation
            scores = cross_val_score(estimator, X, y, cv=5, scoring="f1")
            cv_scores[alg_name] = scores

            print(f"  • {alg_name}: {scores.mean():.4f} ± {scores.std():.4f}")

        except Exception as e:
            print(f"  ❌ {alg_name}: CV failed ({str(e)})")

    # Pairwise statistical tests
    if len(cv_scores) > 1:
        print("\n📊 Pairwise Wilcoxon Signed-Rank Tests:")

        algorithm_names = list(cv_scores.keys())
        significance_results = []

        for i, alg1 in enumerate(algorithm_names):
            for alg2 in algorithm_names[i + 1 :]:
                try:
                    statistic, p_value = wilcoxon(cv_scores[alg1], cv_scores[alg2])
                    is_significant = p_value < 0.05

                    better_alg = (
                        alg1
                        if np.mean(cv_scores[alg1]) > np.mean(cv_scores[alg2])
                        else alg2
                    )

                    result = {
                        "Comparison": f"{alg1} vs {alg2}",
                        "P-value": p_value,
                        "Significant": "✅" if is_significant else "❌",
                        "Better Algorithm": better_alg,
                    }

                    significance_results.append(result)

                    significance_symbol = (
                        "***"
                        if p_value < 0.001
                        else "**"
                        if p_value < 0.01
                        else "*"
                        if p_value < 0.05
                        else "ns"
                    )
                    print(
                        f"  • {alg1} vs {alg2}: p = {p_value:.4f} ({significance_symbol}) - {better_alg} performs better"
                    )

                except Exception as e:
                    print(f"  ❌ {alg1} vs {alg2}: Test failed ({str(e)})")

        # Overall significance test
        if len(cv_scores) > 2:
            try:
                scores_matrix = [cv_scores[alg] for alg in algorithm_names]
                statistic, p_value = friedmanchisquare(*scores_matrix)

                print("\n🔬 Friedman Test (Overall):")
                print(f"  • Statistic: {statistic:.4f}")
                print(f"  • P-value: {p_value:.4f}")

                if p_value < 0.05:
                    print("  • ✅ Significant differences detected between algorithms")
                else:
                    print("  • ❌ No significant differences between algorithms")

            except Exception as e:
                print(f"  ❌ Friedman test failed: {str(e)}")

        # Create significance summary table
        if significance_results:
            significance_df = pd.DataFrame(significance_results)
            print("\n📋 Statistical Significance Summary:")
            display(significance_df)

else:
    print("⚠️ Statistical testing requires ground truth and multiple algorithms")

## 📊 Final Results Summary and Recommendations

In [ ]:
print("=== EXPERIMENT SUMMARY AND RECOMMENDATIONS ===")
print(f"📅 Experiment completed: {datetime.now()}")
print(f"📊 Dataset: {X.shape[0]:,} samples, {X.shape[1]} features")
print(f"🤖 Algorithms tested: {len(ALGORITHMS)}")
print(f"✅ Successful runs: {len(successful_results)}")

if y is not None and not performance_df.empty:
    print("\n🏆 BEST PERFORMING ALGORITHMS:")

    # Sort by F1-score
    top_performers = performance_df.nlargest(3, "F1-Score")

    for i, (_, row) in enumerate(top_performers.iterrows(), 1):
        print(f"  {i}. {row['Algorithm']}:")
        print(f"     • F1-Score: {row['F1-Score']:.4f}")
        print(f"     • Precision: {row['Precision']:.4f}")
        print(f"     • Recall: {row['Recall']:.4f}")
        print(f"     • ROC-AUC: {row['ROC-AUC']:.4f}")
        print(f"     • Execution Time: {row['Execution Time (s)']:.2f}s")

    print("\n💡 RECOMMENDATIONS:")

    best_algorithm = top_performers.iloc[0]["Algorithm"]
    best_f1 = top_performers.iloc[0]["F1-Score"]

    print(f"  • 🥇 Primary recommendation: {best_algorithm}")
    print(f"    - Achieved best F1-score of {best_f1:.4f}")

    # Performance-based recommendations
    if best_f1 > 0.8:
        print("    - ✅ Excellent performance (F1 > 0.8) - ready for production")
    elif best_f1 > 0.6:
        print("    - ⚠️ Good performance (F1 > 0.6) - consider hyperparameter tuning")
    else:
        print(
            "    - 🔧 Moderate performance (F1 < 0.6) - requires further optimization"
        )

    # Speed recommendations
    fastest_alg = performance_df.loc[performance_df["Execution Time (s)"].idxmin()]
    if fastest_alg["Algorithm"] != best_algorithm:
        print(f"  • ⚡ For speed-critical applications: {fastest_alg['Algorithm']}")
        print(f"    - Fastest execution time: {fastest_alg['Execution Time (s)']:.2f}s")
        print(f"    - F1-Score: {fastest_alg['F1-Score']:.4f}")

    # Balance recommendation
    performance_df["efficiency_score"] = (
        performance_df["F1-Score"] / performance_df["Execution Time (s)"]
    )
    most_efficient = performance_df.loc[performance_df["efficiency_score"].idxmax()]

    print(f"  • ⚖️ Best performance/speed balance: {most_efficient['Algorithm']}")
    print(f"    - Efficiency score: {most_efficient['efficiency_score']:.4f}")

else:
    print("\n📊 UNSUPERVISED ANALYSIS SUMMARY:")
    if not unsupervised_df.empty:
        for _, row in unsupervised_df.iterrows():
            print(
                f"  • {row['Algorithm']}: {row['Anomalies Detected']} anomalies ({row['Anomaly Rate']:.1%})"
            )

    print("\n💡 GENERAL RECOMMENDATIONS:")
    print("  • Collect ground truth labels for supervised evaluation")
    print("  • Consider domain expert validation of detected anomalies")
    print("  • Implement ensemble methods for improved robustness")

print("\n🔄 NEXT STEPS:")
print("  1. 🎯 Hyperparameter tuning on best-performing algorithm")
print("  2. 🧪 Feature engineering and selection")
print("  3. 🔗 Ensemble method development")
print("  4. 📈 Cross-validation with larger datasets")
print("  5. 🚀 Production deployment planning")

print("\n📋 EXPERIMENT METADATA:")
print(f"  • Random seed: {RANDOM_STATE}")
print(f"  • Contamination rate: {CONTAMINATION_RATE}")
print(f"  • Total execution time: {sum(execution_times.values()):.2f}s")
print("  • Reproducible: ✅ (random seed set)")

## 💾 Save Results and Export

In [ ]:
# Save experiment results
experiment_results = {
    "metadata": {
        "experiment_date": datetime.now().isoformat(),
        "dataset_shape": X.shape,
        "algorithms_tested": list(ALGORITHMS.keys()),
        "contamination_rate": CONTAMINATION_RATE,
        "random_state": RANDOM_STATE,
    },
    "performance_metrics": performance_df.to_dict("records")
    if y is not None and not performance_df.empty
    else None,
    "execution_times": execution_times,
    "successful_algorithms": list(successful_results.keys()),
    "failed_algorithms": [
        name for name, result in results.items() if "error" in result
    ],
}

# Save to JSON
output_file = f"experiment_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

try:
    import json

    with open(output_file, "w") as f:
        json.dump(experiment_results, f, indent=2, default=str)
    print(f"💾 Results saved to: {output_file}")
except Exception as e:
    print(f"❌ Failed to save results: {str(e)}")

# Export performance summary to CSV
if y is not None and not performance_df.empty:
    csv_file = f"performance_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    try:
        performance_df.to_csv(csv_file, index=False)
        print(f"📊 Performance summary saved to: {csv_file}")
    except Exception as e:
        print(f"❌ Failed to save CSV: {str(e)}")

print("\n✅ Experiment completed successfully!")
print("📋 Check the output files for detailed results and further analysis.")

---

## 📝 Experiment Notes

**Add your observations, insights, and future work ideas here:**

### Key Findings
- [Add your key findings here]
- [Any unexpected results or patterns]
- [Algorithm-specific observations]

### Lessons Learned
- [What worked well]
- [What could be improved]
- [Data quality insights]

### Future Work
- [Planned improvements]
- [Additional algorithms to test]
- [Feature engineering ideas]
- [Production deployment considerations]

---

**🎯 Template Usage**: This template provides a comprehensive framework for anomaly detection experiments. Customize the sections based on your specific needs and domain requirements.

**📚 Documentation**: For more information about Pynomaly, visit the [documentation](https://github.com/your-org/pynomaly) or check the examples directory.

**🤝 Support**: If you encounter issues or have questions, please create an issue in the GitHub repository or contact the development team.